# The key concept to master LCEL (runnable execution order)

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Legacy Chain vs. LCEL Chain

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

### Legacy Chain

In [7]:
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

traditional_chain = LLMChain(
    llm=model,
    prompt=prompt
)

traditional_chain.predict(soccer_player="Maradona")

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv061224-p1/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


'One curious fact about Maradona is that he had a pet dog named "Cachito" that he would often bring to training sessions and games with him. Cachito became a beloved mascot for Maradona and his team, and was even known to chase after the ball during matches.'

### New LCEL Chain

In [8]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano Ronaldo is that he is known for his incredible work ethic and dedication to his fitness. He reportedly has a biological age of 23, despite being in his mid-30s, demonstrating his commitment to maintaining peak physical condition.'

## Let's see this graphically

![alt text](lcel-2.png)


## Replicate this process without using the LCEL chain

### First, we apply .invoke() with user input to prompt template

In [9]:
prompt.invoke({"soccer_player": "Ronaldo"})

ChatPromptValue(messages=[HumanMessage(content='tell me a curious fact about Ronaldo')])

### Then, with completed prompt, we apply .invoke() to model

In [10]:
from langchain_core.messages.human import HumanMessage

output_after_first_step = [HumanMessage(content='tell me a curious fact about Ronaldo')]

In [12]:
model.invoke(output_after_first_step)

AIMessage(content='Ronaldo is known for his incredible work ethic and dedication to training, often spending extra hours perfecting his skills on the field. He is said to have a training routine that includes up to 3,000 sit-ups per day.', response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 14, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9055f0ca-b5f7-4da7-a79b-99d7e47bb00a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 48, 'total_tokens': 62})

### And finally, we apply .invoke() to output parser with output of model

In [13]:
from langchain_core.messages.ai import AIMessage

output_after_second_step = AIMessage(content='One curious fact about Cristiano Ronaldo is that he does not have any tattoos on his body. Despite the fact that many professional athletes have tattoos, Ronaldo has chosen to keep his body ink-free.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 14, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9812511-043a-458a-bfb8-005bc0d057fb-0', usage_metadata={'input_tokens': 14, 'output_tokens': 38, 'total_tokens': 52})

In [14]:
output_parser.invoke(output_after_second_step)

'One curious fact about Cristiano Ronaldo is that he does not have any tattoos on his body. Despite the fact that many professional athletes have tattoos, Ronaldo has chosen to keep his body ink-free.'

## Ways to execute Runnables

#### LCEL Chains/Runnables are used with:
* chain.invoke(): call chain on input
* chain.stream(): call chain on input and stream back chunks of response
* chain.batch(): call chain on list of inputs

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Tell me one sentence about {politician}.")
chain = prompt | model

In [18]:
chain.invoke({"politician": "Churchill"})

AIMessage(content='Winston Churchill was a British statesman who served as Prime Minister during World War II.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 14, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8ca53718-b566-4a64-be08-677702e4e887-0', usage_metadata={'input_tokens': 14, 'output_tokens': 18, 'total_tokens': 32})

In [19]:
for s in chain.stream({"politician": "F.D. Roosevelt"}):
    print(s.content, end="", flush=True)

F.D. Roosevelt was the 32nd President of the United States and served during a time of great economic hardship and world conflict.

In [20]:
chain.batch([{"politician": "Lenin"}, {"politician": "Stalin"}])

[AIMessage(content='Lenin was a Russian revolutionary and politician who led the Bolshevik party to power during the October Revolution of 1917.', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 14, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0d5ae3bf-268e-4c7b-bae0-53991c88bdc9-0', usage_metadata={'input_tokens': 14, 'output_tokens': 24, 'total_tokens': 38}),
 AIMessage(content='Stalin was a ruthless dictator responsible for millions of deaths in the Soviet Union.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 14, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b5e99220-7665-49a4-a124-653027e78b8f-0', usage_metadata={'input_tokens': 14, 'output_tokens': 16, 'total_tokens': 30})]

#### LCEL Chains/Runnables can be also used asynchronously:
* chain.ainvoke(): call chain on input
* chain.astream(): call chain on input and stream back chunks of response
* chain.abatch(): call chain on list of inputs